# Marketing Growth Analysis - Mini Conversion Funnel 

### We want to build a mini conversion funnel from a Landing Page ('lander-1') to Shopping Cart ('/cart')
### We want to know how many people reach each step in their Customer Journey, and drop off  rates
### We are focussing at customer who are looking at product "Mr Fuzzy only" (i.e "/the-original-mr-fuzzy")


In [2]:
# importing Libraries
import os
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error
#import psycopg2
#!pip install pyodbc

## CSV

### CSV Input

In [3]:
df = pd.read_csv('website_pageviews.csv',header=0)
df.head()

,website_pageview_id,created_at,website_session_id,pageview_url
0,1,2012-03-19 08:04:16,1,/home
1,2,2012-03-19 08:16:49,2,/home
2,3,2012-03-19 08:26:55,3,/home
3,4,2012-03-19 08:37:33,4,/home
4,5,2012-03-19 09:00:55,5,/home


### CSV Output

## Create a SQL Lite In Memory DB Connection

In [5]:

import sqlite3
from sqlite3 import Error
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [13]:
conn=create_connection(':memory:')
#website_sessions_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("website_sessions.csv")
website_pageviews_df = pd.read_csv('website_pageviews.csv',header=0)
website_pageviews_df.head()
website_sessions_df = pd.read_csv('website_sessions.csv',header=0)
website_sessions_df.head()
#website_sessions_df = pd.read_csv('website_pageviews.csv')
cursor = conn.cursor()
tbl_name1= "website_pageviews"
cursor.execute("drop table if exists %s;"%(tbl_name1))
#print('Table {0} was created successfully'.format(tbl_name1))
website_pageviews_df.to_sql(tbl_name1,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name1))
tbl_name2= "website_sessions"
cursor.execute("drop table if exists %s;"%(tbl_name2))
#print('Table {0} was created successfully'.format(tbl_name2))
website_sessions_df.to_sql(tbl_name2,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name2))
conn.commit()
#cursor=conn.cursor()
sql="select * from %s limit 10"%tbl_name1
sql_df1 = pd.read_sql('select * from website_sessions limit 3',con=conn)
sql_df2 = pd.read_sql('select * from website_pageviews limit 3',con=conn)
sql_df1.head(3)
#sql_df2.head(3)


File website_pageviews copied to db
File website_sessions copied to db


,website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
0,1,2012-03-19 08:04:16,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
1,2,2012-03-19 08:16:49,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
2,3,2012-03-19 08:26:55,3,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com


                                            ##CONVERSION FUNNEL ANALYSIS
* Its about understanding and Optimizing each step of the User Experience on their Journbey towards purchase of our product 
* Use Case 1- Identifying the most common Paths customers take before purchasing our product 
* Use Case 2- Identifying how many of our users continue on to each next step in conversion flow & how mnay abandon at each step
* Use Case 3-  Optimizing critical pain points where users are abandoning so that you can convert more users & sell more products 
    * Example 95 users  visisted home page, 65 of them went to products page, 45 of the users went to add to cart page for final purchase

In [25]:

session_level_made_it_flag_demo = pd.read_sql("""
select website_session_id, 
max(products_page) products_made_it,
max(mrfuzzy_page) mrfuzzy_made_it,
max(cart_page) cart_page_made_it
from 
(
select website_sessions.website_session_id, 
website_pageviews.pageview_url, 
website_pageviews.created_at pageview_created_at,
case when website_pageviews.pageview_url= '/products' then 1 else 0 end as products_page,
case when website_pageviews.pageview_url= '/the-original-mr-fuzzy' then 1 else 0 end as mrfuzzy_page,
case when website_pageviews.pageview_url= '/cart' then 1 else 0 end as cart_page
from website_sessions left join website_pageviews
on website_sessions.website_session_id = website_pageviews.website_session_id
where 
	 --website_pageviews.pageview_url = (select min(pageview_url) from website_pageviews group by website_session_id)
	--website_sessions.created_at>='2012-03-19 08:26:55' and 
website_pageviews.pageview_url in ('/the-original-mr-fuzzy','/lander-2','/products','/cart') 
order by website_sessions.website_session_id
)x	
group by website_session_id
order by website_session_id
"""
,con=conn)
tbl_name3= "session_level_made_it_flag_demo"
session_level_made_it_flag_demo.to_sql(tbl_name3,conn,if_exists='replace', index=False)

In [28]:
pd.read_sql("""
select count(distinct website_session_id) as sessions,
count(distinct case when products_made_it=1 then website_session_id else null end )*1.0/
count(distinct website_session_id) as lander_click_through_rate , 
count(distinct case when mrfuzzy_made_it =1 then website_session_id else null end )*1.0/
count(distinct website_session_id) as products_through_rate,
ROUND(count(distinct case when cart_page_made_it =1 then website_session_id else null end )*1.0/
count(distinct website_session_id),2) as mrfuzzy_click_through_rate
from session_level_made_it_flag_demo
"""
,con=conn).head()

,sessions,lander_click_through_rate,products_through_rate,mrfuzzy_click_through_rate
0,320480,0.815124,0.50713,0.3


# Great Job!